<a href="https://colab.research.google.com/github/Guiniel/Pasantia-IA-CNTM/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Realizamos las importaciones necesarias

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,Bidirectional,Dropout, GlobalMaxPooling1D, Conv1D, AveragePooling1D
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.layers import Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import preprocessing
from sklearn.metrics import *
from sklearn.model_selection import KFold

Cargamos un archivo que contiene unicamente una columna, dicha columna esta compuesta de requerimientos de software





In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/reqTxt.csv', header=None)
dfRequire = df.iloc[:,:]
print(dfRequire.shape)
print(dfRequire.columns)
X = dfRequire[0]
print(X[0])
X = np.array(X)

(23313, 1)
Int64Index([0], dtype='int64')
add ca against object literals in function invocationsthe idea here is that if our metadata captures a type as function arg we should be able to create an instance of that type as an object literal as an arg to a function invocation for examplep    tiuicreatelabel  ltpropertycaheregt  code prediv


Aqui se carga el conjunto de datos que corresponde a los puntos de esfuerzo de las tareas que se cargaron previamente

In [ ]:
print(len(X))
y = pd.read_csv('/content/drive/MyDrive/data/estiDeep.data', header=None)
y = np.array(y)
print ('Shape of label tensor:', y.shape)
print(y.dtype)


MAX_LEN = 23313 # Cantidad total de registros

kf = KFold(n_splits=10, shuffle=True, random_state=1000)
kf.get_n_splits(X)
print(kf)
print(y[:500])

23313
Shape of label tensor: (23313, 1)
int64
KFold(n_splits=10, random_state=1000, shuffle=True)
[[ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [13]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 8]
 [ 1]
 [ 1]
 [ 1]
 [20]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [13]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 8]
 [ 1]
 [ 1]
 [ 1]
 [ 5]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 8]
 [ 1]
 [ 1]
 [ 1]
 [ 5]
 [ 5]
 [ 1]
 [ 1]
 [ 1]
 [ 8]
 [ 3]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [13]
 [13]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 8]
 [ 8]
 [ 1]
 [ 1]
 [13]
 [ 8]
 [ 1]
 [ 1]
 [ 8]
 [13]
 [ 1]
 [ 1]
 [ 3]
 [ 1]
 [ 1]
 [ 5]
 [21]
 [ 8]
 [ 1]
 [13]
 [ 8]
 [ 5]
 [ 1]
 [ 1]
 [ 1]
 [ 5]
 [13]
 [ 8]
 [ 1]
 [

Aqui cargamos lo llamado embedding pre-entrenado. Esto consiste en **representaciones vectoriales de palabras o frases que se han entrenado previamente en grandes conjuntos de datos textuales**, como corpus de noticias, libros o documentos web. La idea principal detrás de los embeddings pre-entrenados es capturar el significado de las palabras en un espacio vectorial continuo donde palabras con significados similares están más cerca entre sí en este espacio.

In [ ]:
# Cargamos el pre-entrenamiento
pret_model = pd.read_csv('/content/drive/My Drive/pretrain_model/word2vec_SE.csv', sep= ',', header=None)


Aqui entonces, creamos nuestra matriz la cual sirve de modelo preentrenado para proseguir al siguiente paso

In [ ]:
embedding_matrix = pret_model.iloc[1:,:]

dfEmbedding_mat = pd.DataFrame(embedding_matrix)
embedding_mat = dfEmbedding_mat.fillna('0')

print('Embedding mat: ' + str(embedding_mat.shape))

vetMAE = []
vetR2 = []
vetMSE = []
vetMdae = []
i = 1

Embedding mat: (23313, 100)


In [ ]:
for train_index, test_index in kf.split(X):

    x_train, test_x = X[train_index], X[test_index]
    train_y, test_y = y[train_index], y[test_index]


    texts_train = x_train.astype(str)
    texts_test = test_x.astype(str)

    tokenizer = Tokenizer(num_words = MAX_LEN, char_level=False, lower=False)
    tokenizer.fit_on_texts(texts_train)
    encSequences = tokenizer.texts_to_sequences(texts_train)
    encSequences_test = tokenizer.texts_to_sequences(texts_test)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    vocab_size = len(tokenizer.word_index) + 1
    print('Vocab_size: '+ str(vocab_size))

    MAX_SEQUENCE_LENGTH = 100

    x_train = pad_sequences(encSequences, maxlen= MAX_SEQUENCE_LENGTH, padding='post')
    x_test = pad_sequences(encSequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    print('Shape of data tensor:', x_train.shape)
    print('Shape of data test tensor:', x_test.shape)

    print('train_y: ' + str(train_y.shape))
    print('test_y: ' + str(test_y.shape))


    #Sequential model
    model = Sequential()

    embedding = Embedding(MAX_LEN, 100, input_length = 100, name='embedding')
    embedding.build(input_shape=(1,))
    embedding.set_weights([embedding_mat])
    model.add(embedding)

    model.add(AveragePooling1D(pool_size=100))
    print(model.output_shape)

    model.add(Flatten())

    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))

    adam = Adam(lr = 0.001, beta_1 = 0.99, beta_2 = 0.999, epsilon = None, amsgrad = False)

    model.compile(loss = 'mse', optimizer= 'adam', metrics=['mae'])

    model.summary()

    print('Modelo compilado...')

    es = EarlyStopping(monitor='val_mae', mode='min', verbose=1, patience=10, restore_best_weights=True)

    model_history = model.fit(x_train, train_y,
              batch_size= 128,
              epochs=30, callbacks=[es],
              validation_data=(x_test, test_y))

    y_pred = model.predict(x_test, batch_size=None, verbose=0, steps=None)
    x_pred = model.predict(x_train, batch_size=None, verbose=0, steps=None)


    #Metricas
    print("\n")
    mae = mean_absolute_error(test_y, y_pred)
    vetMAE.append(mae)
    print("MAE: %f" % (mae))
    medAE = median_absolute_error(test_y, y_pred)
    vetMdae.append(medAE)
    print("MedAE: %f" % (medAE))
    r2 = r2_score(test_y, y_pred, multioutput='raw_values')
    vetR2.append(r2)
    print("r2: %f" % (r2))
    mse = mean_squared_error(test_y, y_pred)
    vetMSE.append(mse)
    print("MSE: %f" % (mse))
    mErr = max_error(test_y, y_pred)
    print("maxrror: %f" % (mErr))

    i = i + 1
    print("Final " + str(i))

Found 89802 unique tokens.
Vocab_size: 89803
Shape of data tensor: (20981, 100)
Shape of data test tensor: (2332, 100)
train_y: (20981, 1)
test_y: (2332, 1)


(None, 1, 100)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          2331300   
                                                                 
 average_pooling1d (Average  (None, 1, 100)            0         
 Pooling1D)                                                      
                                                                 
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 50)                5050      
                                                                 
 dense_1 (Dense)             (None, 10)                510       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                         

<ipython-input-6-a8e0a612ed1b>:79: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("r2: %f" % (r2))


Found 89519 unique tokens.
Vocab_size: 89520
Shape of data tensor: (20981, 100)
Shape of data test tensor: (2332, 100)
train_y: (20981, 1)
test_y: (2332, 1)


(None, 1, 100)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          2331300   
                                                                 
 average_pooling1d_1 (Avera  (None, 1, 100)            0         
 gePooling1D)                                                    
                                                                 
 flatten_1 (Flatten)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 50)                5050      
                                                                 
 dense_4 (Dense)             (None, 10)                510       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                       

...

(None, 1, 100)
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          2331300   
                                                                 
 average_pooling1d_8 (Avera  (None, 1, 100)            0         
 gePooling1D)                                                    
                                                                 
 flatten_8 (Flatten)         (None, 100)               0         
                                                                 
 dense_24 (Dense)            (None, 50)                5050      
                                                                 
 dense_25 (Dense)            (None, 10)                510       
                                                                 
 dense_26 (Dense)            (None, 1)                 11        
                                       

<ipython-input-6-a8e0a612ed1b>:79: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("r2: %f" % (r2))


Found 89471 unique tokens.
Vocab_size: 89472
Shape of data tensor: (20982, 100)
Shape of data test tensor: (2331, 100)
train_y: (20982, 1)
test_y: (2331, 1)


(None, 1, 100)
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          2331300   
                                                                 
 average_pooling1d_9 (Avera  (None, 1, 100)            0         
 gePooling1D)                                                    
                                                                 
 flatten_9 (Flatten)         (None, 100)               0         
                                                                 
 dense_27 (Dense)            (None, 50)                5050      
                                                                 
 dense_28 (Dense)            (None, 10)                510       
                                                                 
 dense_29 (Dense)            (None, 1)                 11        
                                       

<ipython-input-6-a8e0a612ed1b>:79: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("r2: %f" % (r2))


In [ ]:
maeMedio = np.mean(vetMAE)
madAEMedio = np.mean(vetMdae)
r2Medio = np.mean(vetR2)
mseMedio = np.mean(vetMSE)
stdMae = np.std(vetMAE)
stdr2 = np.std(vetR2)
stdMse = np.std(vetMSE)

print('maeMedio: ' + str(maeMedio))
print('madAEMedio: ' + str(madAEMedio))
print('r2Medio: ' + str(r2Medio))
print('mseMedio: ' + str(mseMedio))
print('stdMae: ' + str(stdMae))
print('stdr2: ' + str(stdr2))
print('stdMse: ' + str(stdMse))

maeMedio: 3.9932860960588137
madAEMedio: 2.075749483704567
r2Medio: 0.2356186116480537
mseMedio: 76.47926138071598
stdMae: 0.1666385134552769
stdr2: 0.04755886124402499
stdMse: 8.820021580146916
